# MC_Issues framework demo

In [12]:
%pip install -q -e  ../. 
from directory_issues.issues.test_issues import IntIssue, StringIssue

from random import choice
from string import ascii_uppercase


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## use metric_type.calculate_all(payload) get all issues found with a payload

In [3]:
refs = []
for i in range(10):
    s = ''.join(choice(ascii_uppercase) for i in range(choice(list(range(5,15)))))
    r = StringIssue.calculate_all(s)
    refs.append(r)
    if len(r) != 0:
        print("\n".join(r_["template"] for r_ in r))
    else:
        print(f'No issues for {s}')
    print("#"*10)

ZKBFLM is too short
ZKBFLM does not contain the letter 'a'
##########
WWZZKVFSGASQAK is too long
WWZZKVFSGASQAK contains duplicate letters
##########
UCQNYHXIBG does not contain the letter 'a'
##########
NAOZWXKKC is too short
NAOZWXKKC contains duplicate letters
##########
GXYBF is too short
GXYBF does not contain the letter 'a'
##########
ZBOEC is too short
ZBOEC does not contain the letter 'a'
##########
HDCNH is too short
HDCNH contains duplicate letters
HDCNH does not contain the letter 'a'
##########
QGGTAZT is too short
QGGTAZT contains duplicate letters
##########
RLKKYSXSMY contains duplicate letters
RLKKYSXSMY does not contain the letter 'a'
##########
DOVRXVI is too short
DOVRXVI contains duplicate letters
DOVRXVI does not contain the letter 'a'
##########


## Use include_tags to only run on certain kinds of issue

In [4]:
for i in range(10):
    s = ''.join(choice(ascii_uppercase) for i in range(choice(list(range(5,15)))))
    r = StringIssue.calculate_all(s, include_tags=["letter"])
    if len(r) != 0:
        print("\n".join(r_["template"] for r_ in r))
    else:
        print(f'No issues for {s}')
    print("#"*10)

FREZINYCZFA contains duplicate letters
##########
No issues for WKAXJFG
##########
HBEKB contains duplicate letters
HBEKB does not contain the letter 'a'
##########
LZBHRQQWDZ contains duplicate letters
LZBHRQQWDZ does not contain the letter 'a'
##########
PGQTXBEQHZALO contains duplicate letters
##########
XCEKIL does not contain the letter 'a'
##########
FTHYQANFAGOOJR contains duplicate letters
##########
OTFTQOFDQJ contains duplicate letters
OTFTQOFDQJ does not contain the letter 'a'
##########
GFHGJL contains duplicate letters
GFHGJL does not contain the letter 'a'
##########
EJEKAMBBL contains duplicate letters
##########


## And use exclude_tags to limit what kinds of issues you're checking for

In [5]:
for i in range(10):
    s = ''.join(choice(ascii_uppercase) for i in range(choice(list(range(5,15)))))
    r = StringIssue.calculate_all(s, exclude_tags=["letter"])
    if len(r) != 0:
        print("\n".join(r_["template"] for r_ in r))
    else:
        print(f'No issues for {s}')
    print("#"*10)

MBYXDXJE is too short
##########
UANXDRQS is too short
##########
No issues for MEXMXOSBIP
##########
QRRZQEKJEDUYCD is too long
##########
LDDUJ is too short
##########
PTGORCFST is too short
##########
OTWKVZNB is too short
##########
No issues for TCZXUAATRA
##########
EOJXTB is too short
##########
ERYQKVOO is too short
##########


## The framework tries to catch and report errors in issue calculation gracefully
Note that execution isn't halted- we just log an error, and report that the issue didn't calculate right. 

In [6]:
print(IntIssue.calculate_all(2, exclude_tags=["breaking"]))
print(IntIssue.calculate_all(3))
print(IntIssue.calculate_all(8, exclude_tags=["breaking"]))

ERROR:directory_issues.issues:Error calculating issue broken_issue for payload 3: Intentional Error
Traceback (most recent call last):
  File "/Users/pgulley/Projects/directory-issues/directory_issues/issues/__init__.py", line 62, in calculate_all
    is_issue, result_data = issue_instance.calculate(payload)
                            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/pgulley/Projects/directory-issues/directory_issues/issues/test_issues.py", line 70, in calculate
    raise RuntimeError("Intentional Error")
RuntimeError: Intentional Error


[{'issue_name': 'is_even', 'template': '2 is even', 'tags': []}]
[{'issue_name': 'broken_issue', 'tags': ['breaking'], 'error': True, 'error_message': 'Intentional Error', 'template': "An error occurred while calculating 'broken_issue'"}]
[{'issue_name': 'is_even', 'template': '8 is even', 'tags': []}]


# Usage:

We shouldn't need very many extensions on the base issue class- we'll certainly need a SourceIssue, FeedIssue, and a CollectionIssue but that's probably it. 
Regardless, generating one is a two-liner: 

In [8]:
from directory_issues.issues import IssueBase
from typing import TypeVar, Tuple, Generic, Type, Dict, Callable, List, Any

class FloatIssue(IssueBase[float]):
    _ISSUES: Dict[str, Type["FloatIssue"]] = {}


Then actually implimenting the issue cases requires creating a class with just two methods- `calculate(payload)` and `render_template()`, then decorating it correctly:

In [9]:
@FloatIssue.register("decimal_places", tags=['decimal'])
class DecimalPlaces(FloatIssue):
    def calculate(self, payload:float) -> Tuple[bool, Dict]:        
        """
        NB the return signature: Tuple[bool, Any]
        bool is ultimately interpreted as "is_issue" - if false, the result is skipped
        Dict is the input provided to the render template
        """
        num, dec = str(payload).split(".")
        exp = len(dec)
    
        return exp > 2, {'value':payload, "exp":exp}

    def render_template(self):
        return f"{self.result['value']} has too many decimal places ({self.result['exp']})"


@FloatIssue.register("negative")
class NegativeFloat(FloatIssue):
    def calculate(self, payload:float):
        return payload < 0, {'value':payload}

    def render_template(self):
        return f"{self.result['value']} is negative"
            

In [10]:
for i in (1.2, -4.7 ,1.24525):
    r = FloatIssue.calculate_all(i)
    print(r)

[]
[{'issue_name': 'negative', 'template': '-4.7 is negative', 'tags': []}]
[{'issue_name': 'decimal_places', 'template': '1.24525 has too many decimal places (5)', 'tags': ['decimal']}]


At dev time, you should only be calling an issue from the calculate_all method- there's no need to ever call issue.calculate or issue.render_template directly. 
